In [1]:
## IF YOU WANT TO RUN THE PROTOCOL, PLEASE READ THE README DOCUMENT ON THE SAME REPOSITORY AS THE PROTOCOL AND FOLLOW THE INSTRUCTIONS
##THIS CODE WILL HELP YOU SIMULATE STOCK SERIAL DILUTION OF INDUCERS   
from opentrons import protocol_api
from opentrons import simulate
from math import ceil

metadata = {'apiLevel': '2.8'}
protocol = simulate.get_protocol_api('2.8')

# Define constants to use throughout protocol
FALCON_TUBE_PBS_VOL = 50000  # Max volume for a falcon tube (50ml)
TIP_RACK_MULTI_POSITION = 10
TIP_RACK_SINGLE_POSITION = 11
PBS_RACK_POSITIONS = [7, 8, 9]
RESERVOIR_POSITIONS = [4, 5, 6, 1, 2, 3]

# Function for obtaining valid numerical input. If input is out of predefined range/ invalid, error message arises
def get_numeric_input(message, ntype=int, min_value=0, max_value=100):

    is_valid = False
    while not is_valid:
        try:
            result = ntype(input(message))
            if result == 'q': # Users can enter 'q' to exit, otherwise they will keep getting asked for input until they enter something correct.
                raise Exception('"q" was entered to quit')
            if result < min_value or result > max_value:
                print("Invalid value entered. Value must be in the range [{}, {}]".format(min_value, max_value))
            else:
                is_valid = True
        except ValueError:
            print("Invalid value entered. You must enter a {}.".format(ntype.__name__))
    return result

# Function for tracking PBS falcon tubes and their positions
def get_pbs_tube(tube_id, racks):

    rack_id = tube_id // 6 # 6 slots in one rack
    rack_row = ['A', 'B'][(tube_id % 6) // 3] # There are 6 rows on 3 racks, so 2 rows on each rack, called either 'A' or 'B'
    rack_col = (tube_id % 3) + 1 # There are 3 columns per rack
    return racks[rack_id][rack_row + str(rack_col)]

# User input
num_inducers = get_numeric_input("How many inducers are you working with? (max. 6) ", max_value=6)
num_folds = get_numeric_input("What fold dilutions would you want the concentrations of your inducers to be? (enter 2 for 2-fold, 10 for 10-fold, etc.) ", ntype=float, max_value=10)
stockvol = get_numeric_input("Please state the value of the inducer solution volume (in mL) you have put into A1 of each inducer reservoir: (max. 15)", ntype=float, max_value=15) * 1000 # User input in mL for their convenience
num_concs = get_numeric_input("Please state the number of concentrations you would want to have (max. 12): ", max_value=12)

max_falcon_tubes = 6 * len(PBS_RACK_POSITIONS) #where PBS_RACK_POSITIONS = 3
num_falcon_tubes = get_numeric_input("Please state the number of falcon tubes of PBS you have loaded (max. {})".format(max_falcon_tubes), max_value=max_falcon_tubes)

# # Check that there is enough PBS to perform all the dilutions
PBSvol = stockvol/num_folds*(num_folds-1)
total_pbs_required = PBSvol * (num_concs - 1) * num_inducers
if total_pbs_required > num_falcon_tubes * FALCON_TUBE_PBS_VOL:
    raise Exception("ERROR: You don't have enough PBS to complete all dilutions. "
                    "You only have {} falcon tubes of PBS but you need {}"
                    "Please separate the sets of dilutions into different runs of the code."
                    .format(num_falcon_tubes, ceil(total_pbs_required/FALCON_TUBE_PBS_VOL)))

# Start of the protocol:
# Load all the labware and instruments
tip_rack_multi = protocol.load_labware('opentrons_96_tiprack_300ul',
                                       TIP_RACK_MULTI_POSITION) # please commence with full tip rack (especially working with max number of inducers)

tip_rack_single = protocol.load_labware('opentrons_96_tiprack_1000ul',
                                        TIP_RACK_SINGLE_POSITION)

multi = protocol.load_instrument('p300_multi_gen2',
                                 'left',
                                 tip_racks=[tip_rack_multi])

single = protocol.load_instrument('p1000_single_gen2',
                                  'right',
                                  tip_racks=[tip_rack_single])

# Create a list of racks for the PBS falcon tubes via loop
pbs_racks = [protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical', PBS_RACK_POSITIONS[i])
             for i in range(ceil(num_falcon_tubes / 6))] #round up to ensure have enough falcon tubes 

# Create a list of reservoir racks via loop
reservoirs = [protocol.load_labware('nest_12_reservoir_15ml', RESERVOIR_POSITIONS[n])
              for n in range(num_inducers)]

# Start the serial dilutions
current_pbs_tube = 0                        # Falcon tube of PBS we are currently using
current_pbs_volume = FALCON_TUBE_PBS_VOL    # The volume of PBS remaining in the current falcon tube

single.pick_up_tip(tip_rack_single['A1'])

    
# For each reservoir (i.e. inducer), fill wells with PBS
for reservoir in reservoirs:
    # Fill reservoir wells with PBS
    for i in range(1, num_concs):
        reservoir_well = reservoir['A' + str(i+1)]

        # Fill well with PBS, from multiple falcon tubes if necessary
        amount_remaining = PBSvol   # The amount of PBS we still have to transfer to the well
        while amount_remaining > 0:
            pbs_tube = get_pbs_tube(current_pbs_tube, pbs_racks)

            # Check if the current falcon tube has enough PBS to finish filling this well
            if current_pbs_volume > amount_remaining:
                transfer_vol = amount_remaining  # if there is enough, transfer the amount
            else:
                transfer_vol = current_pbs_volume  # if there is not enough, transfer everything that remains in the falcon tube
            
                
            # Perform the PBS transfer                
            single.transfer(transfer_vol, pbs_tube, reservoir_well,
                            touch_tip=False, 
                            blow_out=True, 
                            blowout_location='destination well', 
                            new_tip='never')
            
            
            # Update the amount of PBS in the current falcon tube, and the amount still needed for the well
            current_pbs_volume -= transfer_vol
            amount_remaining -= transfer_vol

            # If the current falcon tube is basically empty...
            if current_pbs_volume < 100: #0.1 uL
                current_pbs_tube += 1  # Switch to the next falcon tube
                current_pbs_volume = FALCON_TUBE_PBS_VOL  # The next falcon tube is full

single.drop_tip()

# For each reservoir, perform the dilution
cap_multi = 300
for n, reservoir in enumerate(reservoirs, start=1):
    # Transferring the amount needed for serial dilution from well (i) to adjacent wells (i+1) in the reservoir
    multi.pick_up_tip(tip_rack_multi['A' + str(n)]) # Pick up a new multi-tip for each inducer

    amt = (stockvol/num_folds) / 8
    for i in range(1, num_concs - 1): #num of concs = num of dilutions + 1
        start_well = reservoir['A' + str(i)]
        end_well = reservoir['A' + str(i+1)]

        if amt > cap_multi:
            rep = int(amt // cap_multi) #num of repeated pipetting needed to transfer vol to be transferred
            remaining = amt % cap_multi #amount of remaining volume to be pipetted for the last transfer

        for j in range(rep):
            multi.aspirate(cap_multi, start_well)
            multi.dispense(cap_multi, end_well)
    
        multi.aspirate(remaining, start_well)
        multi.dispense(remaining, end_well)

        multi.mix(7, cap_multi) #mixing to ensure uniform concentration in each well

    multi.drop_tip()

for line in protocol.commands():
    print(line)

/Users/suhasiniiyer/.opentrons/robot_settings.json not found. Loading defaults
/Users/suhasiniiyer/.opentrons/deck_calibration.json not found. Loading defaults


How many inducers are you working with? (max. 6) 2
What fold dilutions would you want the concentrations of your inducers to be? (enter 2 for 2-fold, 10 for 10-fold, etc.) 2
Please state the value of the inducer solution volume (in mL) you have put into A1 of each inducer reservoir: (max. 15)10
Please state the number of concentrations you would want to have (max. 12): 6
Please state the number of falcon tubes of PBS you have loaded (max. 18)2
Picking up tip from A1 of Opentrons 96 Tip Rack 1000 µL on 11
Transferring 5000.0 from A1 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 7 to A2 of NEST 12 Well Reservoir 15 mL on 4
Aspirating 1000.0 uL from A1 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 7 at 274.7 uL/sec
Dispensing 1000.0 uL into A2 of NEST 12 Well Reservoir 15 mL on 4 at 274.7 uL/sec
Blowing out at A2 of NEST 12 Well Reservoir 15 mL on 4
Aspirating 1000.0 uL from A1 of Opentrons 6 Tube Rack with Falcon 50 mL Conical on 7 at 274.7 uL/sec
Dispensing 1000.0 uL int